In [1]:
import sys
sys.path.append("/home/ubuntu/MultiModalDeepFake")
import nemo.collections.asr as nemo_asr 
import pandas as pd

/home/ubuntu/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (5.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-04-05 05:05:37.860259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 05:05:37.963957: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-05 05:05:38.495498: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.c

In [2]:
from packages.LJDataLoader import LJDataLoader
from packages.AudioEmbeddingsManager import AudioEmbeddingsManager
from packages.ModelManager import ModelManager

## Generate the Train/Dev/Test

In [3]:
file_path = '/home/ubuntu/data/wavefake_data/LJ_metadata_16000KHz.csv'
loader = LJDataLoader(data_path=file_path)
loader.sample(0.1)
loader.splitData()

source_architectures = ['Full_Band_MelGan', 'HifiGan', 'MelGan', 'MelGanLarge', 'Multi_Band_MelGan', 'Parallel_WaveGan', 'Waveglow']
new_col_name = 'RandWaveFake'
loader.selectRandomArchitecture(target_col=new_col_name, source_cols=source_architectures)

#data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['RandWaveFake', 'ElevenLabs', 'UberDuck'])
data_df = loader.generateFinalDataFrame(real_col='Real', fake_cols=['ElevenLabs', 'UberDuck'])

## Generate the Features

In [4]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='titanet_large')

[NeMo I 2023-04-05 05:05:41 cloud:56] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.
[NeMo I 2023-04-05 05:05:41 cloud:62] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo
[NeMo I 2023-04-05 05:05:41 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-04-05 05:05:41 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-04-05 05:05:41 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_valida

[NeMo I 2023-04-05 05:05:41 label_models:126] Setting angular: true/false in decoder is deprecated and will be removed in 1.13 version, use specific loss with _target_
[NeMo I 2023-04-05 05:05:41 features:267] PADDING: 16
[NeMo I 2023-04-05 05:05:42 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.15.0/titanet-l/492c0ab8416139171dc18c21879a9e45/titanet-l.nemo.


In [5]:
embedding_manager = AudioEmbeddingsManager(model=speaker_model, data=data_df)

In [6]:
feature_df, feature_cols = embedding_manager.generateFeatureDf()

[NeMo W 2023-04-05 05:05:43 nemo_logging:349] /home/ubuntu/miniconda3/envs/nemo/lib/python3.8/site-packages/nemo/collections/asr/models/label_models.py:437: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525491394/work/torch/csrc/utils/tensor_new.cpp:230.)
      torch.tensor([audio], device=device),
    


In [7]:
feature_df.head()

type                                               path  label  \
0  train  /home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...      0   
1  train  /home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...      0   
2  train  /home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...      0   
3  train  /home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...      0   
4  train  /home/ubuntu/data/wavefake_data/LJSpeech_1.1/w...      0   

   multiclass_label         0         1         2         3         4  \
0                 0 -0.004833  0.011228 -0.007260 -0.003250 -0.023042   
1                 0 -0.013503 -0.002885  0.003451  0.018167 -0.022606   
2                 0 -0.006981  0.004201 -0.015476  0.031246 -0.012042   
3                 0  0.008485 -0.009508  0.005324  0.023528 -0.014678   
4                 0  0.006824 -0.004262 -0.000653  0.018907 -0.003375   

          5  ...       182       183       184       185       186       187  \
0  0.024393  ...  0.006901  0.004162 -0.012884 -0.014927  0.021247  0.012206   
1  0.020847  ... -0.018954  0.004550  0.010938 -0.013638  0.005500  0.003162   
2  0.005981  ... -0.017657  0.004625 -0.010638 -0.016903  0.013171  0.017300   
3  0.025951  ... -0.040629 -0.014372 -0.014995 -0.003999 -0.001448  0.032389   
4  0.028410  ... -0.017940 -0.008076 -0.000130 -0.009583 -0.001130  0.012930   

        188       189       190       191  
0  0.006459  0.011414  0.007401  0.003593  
1 -0.009371  0.004279  0.019605  0.002401  
2  0.012562  0.000862  0.004463 -0.010272  
3  0.006757 -0.001001  0.010690  0.017595  
4  0.007421  0.009749  0.014425 -0.002132  

[5 rows x 196 columns]

## Predict and Evaluate

In [8]:
model_manager = ModelManager('logreg', feature_df, feature_cols, merge_train_dev=True)

In [9]:
acc, logloss = model_manager.trainPredict(label_col='multiclass_label')

In [10]:
#model_manager.plotRocCurve()

In [11]:
model_manager.plotProbaDistribution()

In [12]:
print(acc, logloss)

1.0 0.20505706174352645
